In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
x_vals_path = os.path.join('x_vals.json')
dummy_x_vals_path = os.path.join('shared/dummy_x_vals.json')
x_open = open(x_vals_path, "r")
x_vals= json.loads(x_open.read())['input_data'][0]
dummy_x_vals = np.random.uniform(min(x_vals), max(x_vals), len(x_vals))
json.dump({"input_data":[dummy_x_vals.tolist()]}, open(dummy_x_vals_path, 'w'))


y_vals_path = os.path.join('y_vals.json')
dummy_y_vals_path = os.path.join('shared/dummy_y_vals.json')
y_open = open(y_vals_path, "r")
y_vals= json.loads(y_open.read())["input_data"][0]
dummy_y_vals = np.random.uniform(min(y_vals), max(y_vals), len(y_vals))
json.dump({"input_data":[dummy_y_vals.tolist()]}, open(dummy_y_vals_path, 'w'))


real_cov = statistics.covariance(x_vals, y_vals)
x_mean = statistics.mean(x_vals)
y_mean = statistics.mean(y_vals)
print("cov: ",real_cov )
print("x mean: ", x_mean)
print("y mean: ", y_mean)

dummy_cov = statistics.covariance(dummy_x_vals, dummy_y_vals)
dummy_x_mean = statistics.mean(dummy_x_vals)
dummy_y_mean = statistics.mean(dummy_y_vals)

cov:  3214.750736446082
x mean:  49.5
y mean:  227.70285322096493


In [9]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.cov = nn.Parameter(data = torch.tensor(dummy_cov), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(dummy_x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(dummy_y_mean), requires_grad = False)
    def forward(self,X,Y):
        # some expression of tolerance to error in the inference
        # print("x size: ", X.size()[1])
        #  need to enforce same length, not yet
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<0.01*X.size()[1]*(self.x_mean)
        y_mean_cons = torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<0.01*Y.size()[1]*(self.y_mean)
        return (torch.logical_and(torch.logical_and(x_mean_cons,y_mean_cons), torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(X.size()[1]-1)*(self.cov))<0.01*(X.size()[1]-1)*(self.cov)), self.cov)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_x_vals_path, dummy_y_vals_path])

In [10]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(real_cov)
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.cov = nn.Parameter(data = torch.tensor(real_cov), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(y_mean), requires_grad = False)
    def forward(self,X,Y):
        # some expression of tolerance to error in the inference
        # print("x size: ", X.size()[1])
        #  need to enforce same length
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<0.01*X.size()[1]*(self.x_mean)
        y_mean_cons = torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<0.01*Y.size()[1]*(self.y_mean)
        return (torch.logical_and(torch.logical_and(x_mean_cons,y_mean_cons), torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(X.size()[1]-1)*(self.cov))<0.01*(X.size()[1]-1)*(self.cov)), self.cov)

prover_gen_settings([x_vals_path, y_vals_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

Theory output:  tensor(3214.7507)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[0,6226],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":13120,"total_assignments":1018,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0,0],"module_sizes":{"kzg":[],"poseidon":[13120,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [11]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.340538740158081 seconds
Theory output:  tensor(3214.7507)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 3215.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[14955570959218682635, 4667139652385906200, 12836539004462631467, 1774684518626433649], [2636517083442646612, 5059672846326347070, 4705835036930416229, 3312472395716484608], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5743885005642251665, 3430503978676436355, 7149667244725939006, 2902673458086333540]]], 'proof': '08b3c2fd4a107bf3e9fe326a09762d80e571071232441ff70777c3b91dc30a3b0c3a9266e2f82f61d93e881fe1b189ea8629665a61dd118eb1c34726c82c9c2408b0449b12d04aa29bad952a9d81ed0fce13306527bda15d4a5e08b8978e4e9e052f104728a45839878420642ca6e7a448798b3abbdec08a94385c22473be31222ed26486712bf5d5612f9dffa980873b16667a9d95c41fc9451d11400158a342d646bd3d6855cad4ff0947f6f8d408018279767ef220f6b1729d069d75ef93f26c4359f5df8c0a761cec0163c8c08e91136bf043ae9335700ddb354d350e3682f299e3a879b8fffd3e3963502d821e0742d025c54b176c4f6a466e6312401671831689b309929d6c6132c92d93daba78f0639c9a8cbbb081b09da5b4709c46705b4ec0e4c3fa20a320bb58173fceb657a661e1ac

In [13]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  2
prf instances:  [[[14955570959218682635, 4667139652385906200, 12836539004462631467, 1774684518626433649], [2636517083442646612, 5059672846326347070, 4705835036930416229, 3312472395716484608], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5743885005642251665, 3430503978676436355, 7149667244725939006, 2902673458086333540]]]
proof boolean:  1.0
proof result 1 : 3215.0
verified
